In [2]:
import os
import warnings
import pandas as pd
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Binary Classification

## Twitter

In [33]:
train_path = '../../../data/moral/interim/train_test/twitter/train_bert_binary.json'
test_path = '../../../data/moral/interim/train_test/twitter/test_bert_binary.json'
output_path = '../../../data/moral/processed/twitter/prompt/baseline-1ex/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(test_path, lines=True)

train_data_1 = train_data[train_data['label'] == 1]
train_data_0 = train_data[train_data['label'] == 0]

prompts_df = pd.DataFrame(columns=['line_idx', 'original_example', 'full_prompt', 'instantiation_prompt'])

for idx, test_sample in test_data.iterrows():
    # randomly sample 1 example from train with label 1 and 1 example from train with label 0
    eg1 = train_data_1.sample(1)['text'].values[0]
    res1 = 1
    eg2 = train_data_0.sample(1)['text'].values[0]
    res2 = 0
    infer = test_sample['text']

    # design prompts for inferring codex
    prompt = f"""determine whether the following sentence presents a moral value or not. Return 1 if it does, 0 if it does not. Don't return anything else.
    Example with moral value: 
    Sentence: {eg1.strip()} 
    Return: {res1}
    Example without moral value:
    Sentence: {eg2.strip()}
    Return: {res2}
    Do Inference:
    Sentence: {infer.strip()}
    Return: """
    
    prompts_df = prompts_df.append({'line_idx': idx, 'original_example': test_sample.to_dict(), 'full_prompt': prompt, 'instantiation_prompt': ''}, ignore_index=True)

    # write prompt to file
    if not os.path.exists(output_path + 'test/'):
        os.makedirs(output_path + 'test/')
    with open(output_path + 'test/' + str(idx) + '.txt', 'w') as f:
        f.write(prompt)
        
# save the prompts df
prompts_df.to_json(output_path + 'test.jsonl', orient='records', lines=True)

## Reddit

In [41]:
train_path = '../../../data/moral/interim/train_test/reddit/train_bert_binary.json'
test_path = '../../../data/moral/interim/train_test/reddit/test_bert_binary.json'
output_path = '../../../data/moral/processed/reddit/prompt/baseline-1ex/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(test_path, lines=True)

train_data_1 = train_data[train_data['label'] == 1]
train_data_0 = train_data[train_data['label'] == 0]

prompts_df = pd.DataFrame(columns=['line_idx', 'original_example', 'full_prompt', 'instantiation_prompt'])

for idx, test_sample in test_data.iterrows():
    # randomly sample 1 example from train with label 1 and 1 example from train with label 0
    eg1 = train_data_1.sample(1)['text'].values[0]
    res1 = 1
    eg2 = train_data_0.sample(1)['text'].values[0]
    res2 = 0
    infer = test_sample['text']

    # design prompts for inferring codex
    prompt = f"""determine whether the following sentence presents a moral value or not. Return 1 if it does, 0 if it does not. Don't return anything else.
    Example with moral value: 
    Sentence: {eg1.strip()} 
    Return: {res1}
    Example without moral value:
    Sentence: {eg2.strip()}
    Return: {res2}
    Do Inference:
    Sentence: {infer.strip()}
    Return: """
    
    prompts_df = prompts_df.append({'line_idx': idx, 'original_example': test_sample.to_dict(), 'full_prompt': prompt, 'instantiation_prompt': ''}, ignore_index=True)

    # write prompt to file
    if not os.path.exists(output_path + 'test/'):
        os.makedirs(output_path + 'test/')
    with open(output_path + 'test/' + str(idx) + '.txt', 'w') as f:
        f.write(prompt)
        
# save the prompts df
prompts_df.to_json(output_path + 'test.jsonl', orient='records', lines=True)


# Multiclass Multilabel classification

## Twitter

In [18]:
train_path = '../../../data/moral/interim/train_test/twitter/train_bert.json'
test_path = '../../../data/moral/interim/train_test/twitter/test_bert.json'
output_path = '../../../data/moral/processed/twitter/prompt/baseline-50exwxp/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(test_path, lines=True)

moral_values = ['care_or_harm', 'fairness_or_cheating', 'loyalty_or_betrayal', 'authority_or_subversion', 'purity_or_degradation', 'non-moral']

num_examples = 50
prompts_df = pd.DataFrame(columns=['line_idx', 'original_example', 'full_prompt', 'instantiation_prompt'])

for idx, test_sample in tqdm(test_data.iterrows()):

    prompt = f"""There are 5 possible moral values present in a sentence, but it's also possible that no moral values are present at all.
If any of the moral values are present, return the list of moral values present in the sentence. If no moral values are present, return ['non-moral'].
The total set of moral values are: {str(moral_values)}
care_or_harm is related to our long evolution as mammals with attachment systems and an ability to feel (and dislike) the pain of others. It underlies virtues of kindness, gentleness, and nurturance.
fairness_or_cheating is related to the evolutionary process of reciprocal altruism. It generates ideas of justice, rights, and autonomy.
loyalty_or_betrayal is related to our long history as tribal creatures able to form shifting coalitions. It underlies virtues of patriotism and self-sacrifice for the group. It is active anytime people feel that it’s “one for all, and all for one.”
authority_or_subversion is related to our long primate history of hierarchical social interactions. It underlies virtues of leadership and followership including deference to legitimate authority and respect for traditions.
purity_or_degradation is shaped by the psychology of disgust and contamination. It underlies religious notions of striving to live in an elevated, less carnal, more noble way. It underlies the widespread idea that the body is a temple that can be desecrated by immoral activities and contaminants (an idea not unique to religious traditions).
Examples:\n"""
    
    # for i in range(num_examples * len(moral_values)):
    #     ex = train_data[train_data[moral_values[i % len(moral_values)]] == 1].sample(1)
    #     present_moral_values = ex.columns[ex.eq(1).any()].tolist()
    #     prompt_slice_i = f"""\nSentence: {ex['text'].values[0].strip()}\nReturn: {str(present_moral_values)}\n"""
    #     prompt += prompt_slice_i
        
    for i in range(num_examples):
        ex = train_data.sample(1)
        present_moral_values = ex.columns[ex.eq(1).any()].tolist()
        prompt_slice_i = f"""\nSentence: {ex['text'].values[0].strip()}\nReturn: {str(present_moral_values)}\n"""
        prompt += prompt_slice_i
        
    prompt += f"""\nPredict:\nSentence: {test_sample['text'].strip()}\nReturn: """

    prompts_df = prompts_df.append({'line_idx': idx, 'original_example': test_sample.to_dict(), 'full_prompt': prompt, 'instantiation_prompt': ''}, ignore_index=True)

    # write prompt to file
    if not os.path.exists(output_path + 'test/'):
        os.makedirs(output_path + 'test/')
    with open(output_path + 'test/' + str(idx) + '.txt', 'w') as f:
        f.write(prompt)
         
prompts_df.to_json(output_path + 'test.jsonl', orient='records', lines=True)

5524it [03:22, 27.29it/s]


## Reddit

In [4]:
train_path = '../../../data/moral/interim/train_test/reddit/train_bert.json'
test_path = '../../../data/moral/interim/train_test/reddit/test_bert.json'
output_path = '../../../data/moral/processed/reddit/prompt/baseline-1expmv/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(test_path, lines=True)

moral_values = ['Purity', 'Loyalty', 'Non-Moral', 'Authority', 'Equality', 'Care', 'Proportionality']

num_examples = 1
prompts_df = pd.DataFrame(columns=['line_idx', 'original_example', 'full_prompt', 'instantiation_prompt'])

for idx, test_sample in tqdm(test_data.iterrows()):
    prompt = f"""There are 6 possible moral values present in a sentence, but it's also possible that no moral values are present at all.
    If any of the moral values are present, return the list of moral values present in the sentence. If no moral values are present, return ['Non-Moral'].
    The total set of moral values are: {str(moral_values)}\n"""

    for i in range(num_examples * len(moral_values)):
        ex = train_data[train_data[moral_values[i]] == 1].sample(1)
        present_moral_values = ex.columns[ex.eq(1).any()].tolist()
        prompt_slice_i = f"""Example with {moral_values[i]} present:\nSentence: {ex['text'].values[0].strip()}\nReturn: {str(present_moral_values)}\n"""
        prompt += prompt_slice_i
        
    prompt += f"""Predict:\nSentence: {test_sample['text'].strip()}\nReturn: """

    prompts_df = prompts_df.append({'line_idx': idx, 'original_example': test_sample.to_dict(), 'full_prompt': prompt, 'instantiation_prompt': ''}, ignore_index=True)

    # write prompt to file
    if not os.path.exists(output_path + 'test/'):
        os.makedirs(output_path + 'test/')
    with open(output_path + 'test/' + str(idx) + '.txt', 'w') as f:
        f.write(prompt)
        
prompts_df.to_json(output_path + 'test.jsonl', orient='records', lines=True)

2689it [00:19, 136.00it/s]
